In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
!pip install faker
!pip install bcrypt
!pip install bson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11975 sha256=9edb26be5fe8a51926c96cb5ece4810f7a5264198e4cbf830232f1f25a5b56d1
  Stored in directory: /root/.cache/pip/wheels/cb/f3/45/c859e83339943dfe2f43e1c9aaebdc00db321191a6fe120947
Successfully built bson


In [4]:
import os
import pandas as pd
import random
from math import floor, radians, sin, cos, sqrt, atan2
import ast
import json
from faker import Faker
from bcrypt import hashpw, gensalt
from bson import ObjectId
from datetime import datetime, timedelta

fake = Faker()

# Function to hash passwords
def hash_password(password: str) -> str:
    return hashpw(password.encode('utf-8'), gensalt(rounds=4)).decode('utf-8')

# Function to calculate the distance between two geographic points (latitude, longitude)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Function to get the nearest neighbourhood
def get_neighbourhood(lat, lon, city):
    city = city.title()
    city_neighbourhoods = neighbourhoods_df[neighbourhoods_df['city'] == city].copy()
    if city_neighbourhoods.empty:
        print(f"No neighbourhood found for city: {city}")
        return ''
    city_neighbourhoods['distance'] = city_neighbourhoods.apply(
        lambda row: haversine(lat, lon, row['latitude'], row['longitude']), axis=1
    )
    nearest_neighbourhood = city_neighbourhoods.loc[city_neighbourhoods['distance'].idxmin()]
    return nearest_neighbourhood['name']

# Function to generate buyers
def generate_buyers(properties_on_sale_flat, output_folder):
    buyers = []
    # Group unsold properties by city
    city_properties = {}
    for prop in properties_on_sale_flat:
        summary = {
            '_id': prop['_id'],
            'thumbnail': prop['thumbnail'],
            'address': prop['address'],
            'price': prop['price'],
            'area': prop['area']
        }
        city = prop['city']
        if city not in city_properties:
            city_properties[city] = []
        city_properties[city].append(summary)

    # For each city, generate a random number of buyers (between 100 and 300)
    for city, props in city_properties.items():
        num_buyers = random.randint(100, 300)
        for _ in range(num_buyers):
            fake_first_name = fake.first_name()
            fake_last_name = fake.last_name()
            buyer = {
                '_id': str(ObjectId()),
                'name': fake_first_name,
                'surname': fake_last_name,
                'email': fake_first_name.lower() + fake_last_name.lower() + '@buyer.com',
                'password': hash_password(fake.password()),
                'phone_number': fake.phone_number(),
                'favourites': []
            }
            # Randomly select between 1 and 3 properties as favourites from the same city
            num_favs = random.randint(1, min(3, len(props))) if props else 0
            if num_favs > 0:
                favs = random.sample(props, num_favs)
                buyer['favourites'] = favs
            buyers.append(buyer)

    # Export buyers to CSV
    buyers_df = pd.DataFrame(buyers)
    buyers_df['favourites'] = buyers_df['favourites'].apply(json.dumps)
    buyers_output = os.path.join(output_folder, 'buyers.csv')
    buyers_df.to_csv(buyers_output, index=False)

# Main function to process CSV files and generate output files
def process_files(input_folder):
    all_properties = []  # List of all properties (flat) for the PropertyOnSale CSV
    sellers = {}         # Dictionary: key = agency_name, value = dict with seller info and a list of properties

    # Process each file properties_*.csv
    for file_name in os.listdir(input_folder):
        if file_name.startswith('properties_') and file_name.endswith('.csv'):
            # Assuming file name format is properties_<city>_<state>.csv
            city, state = file_name.replace('properties_', '').replace('.csv', '').split('_')
            file_path = os.path.join(input_folder, file_name)
            columns_to_load = ['latLong', 'addressCity', 'address', 'unformattedPrice', 'imgSrc',
                                 'statusText', 'area', 'beds', 'baths', 'flexFieldText', 'carouselPhotos',
                                 'brokerName', 'hdpData', 'zestimate', 'hasOpenHouse', 'openHouseDescription']
            df = pd.read_csv(file_path, usecols=columns_to_load)

            for _, row in df.iterrows():
                # Extract coordinates
                lat, lon = (None, None)
                if 'latLong' in row and pd.notna(row['latLong']):
                    try:
                        lat_lon_dict = ast.literal_eval(row['latLong'])
                        if isinstance(lat_lon_dict, dict) and 'latitude' in lat_lon_dict and 'longitude' in lat_lon_dict:
                            lat, lon = lat_lon_dict['latitude'], lat_lon_dict['longitude']
                    except (ValueError, SyntaxError):
                        pass

                # Parse hdpData
                hdp_data = {}
                if 'hdpData' in row and pd.notna(row['hdpData']):
                    try:
                        hdp_data = json.loads(row['hdpData']) if isinstance(row['hdpData'], str) else row['hdpData']
                    except (json.JSONDecodeError, TypeError):
                        pass

                # If neighbourhood is missing, calculate the nearest one
                neighbourhood = None
                if not neighbourhood and lat and lon:
                    neighbourhood = get_neighbourhood(lat, lon, city)

                area_value = row.get('area', 0)
                max_attendees = random.randint(8, 15)

                # Extract agency name from brokerName field
                temp_agency = row.get('brokerName', '')
                if pd.notna(temp_agency):
                    if 'Listing by:' in temp_agency:
                        agency_name = temp_agency.split('Listing by:')[1].strip()
                    else:
                        agency_name = temp_agency.strip()
                else:
                    agency_name = ''

                # Build dynamic description
                description_parts = []
                if 'homeType' in hdp_data:
                    description_parts.append(f"🏡 Property type: {hdp_data['homeType']}")
                if 'livingArea' in hdp_data:
                    description_parts.append(f"📏 Living area: {hdp_data['livingArea']} sqft")
                if 'taxAssessedValue' in hdp_data:
                    description_parts.append(f"💸 Tax assessed value: ${hdp_data['taxAssessedValue']:,}")
                if 'lotAreaValue' in hdp_data:
                    description_parts.append(f"🌳 Lot size: {hdp_data['lotAreaValue']} acres")
                if pd.notna(row.get('zestimate')):
                    description_parts.append(f"💰 Hxestimate: ${row['zestimate']:,}")
                if agency_name != '':
                    description_parts.append(f"👔 Agency: {agency_name}")

                description = "\n".join(description_parts)


                # Set registration_date as current date minus days on Zillow (if available)
                if 'daysOnZillow' in hdp_data:
                    registration_date = (datetime.now() - timedelta(days=hdp_data['daysOnZillow'])).isoformat()
                else:
                    registration_date = datetime.now().isoformat()

                # --- Begin updated open house time formatting code ---
                # Generate a random day of the week for open house
                days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
                open_house_day = random.choice(days_of_week)

                # Retrieve the open house time from the row data
                open_house_time = row.get('openHouseDescription', '')

                # Helper function to generate a random time range (morning or afternoon)
                def random_time_range():
                    period = random.choice(["AM", "PM"])
                    if period == "AM":
                        random_hour = random.randint(8, 11)  # Morning hours between 8 and 11
                        start = f"{random_hour}:00 {period}"
                        end = f"{random_hour + 1}:00 {period}"
                    else:
                        # For PM, choose a random hour between 12 and 18, and convert to 12-hour format if needed
                        random_hour = random.randint(12, 18)
                        display_hour = random_hour if random_hour == 12 else random_hour - 12
                        start = f"{display_hour}:00 {period}"
                        # Ensure proper progression (if start is 11:00 PM, end becomes 12:00 PM, which is acceptable)
                        if display_hour == 11:
                            end = f"12:00 {period}"
                        else:
                            end = f"{display_hour + 1}:00 {period}"
                    return f"{start} - {end}"

                if open_house_time and pd.notna(open_house_time):
                    # Remove any prefixes such as "Open House:" or "Open House -"
                    if 'Open House:' in open_house_time:
                        open_house_time = open_house_time.split('Open House:')[1].strip()
                    elif 'Open House -' in open_house_time:
                        open_house_time = open_house_time.split('Open House -')[1].strip()

                    # Check if the time is provided as a range using a hyphen
                    if '-' in open_house_time:
                        parts = open_house_time.split('-', 1)
                        start_time = parts[0].strip()
                        end_time = parts[1].strip()

                        # If the start time does not contain "AM" or "PM" but the end time does,
                        # append the period from end_time to start_time.
                        if not ("AM" in start_time or "PM" in start_time) and ("AM" in end_time or "PM" in end_time):
                            period = end_time.split()[-1]  # Extract the period ("AM" or "PM") from end_time
                            start_time = f"{start_time} {period}"
                            open_house_time = f"{start_time} - {end_time}"
                        else:
                            # If times are already properly formatted (either morning or afternoon), use them as is.
                            open_house_time = f"{start_time} - {end_time}"
                    else:
                        # If there is no hyphen (i.e., not a range), assign a random time range
                        open_house_time = random_time_range()
                else:
                    # If no open house time is provided, assign a random time range
                    open_house_time = random_time_range()
                # --- End updated open house time formatting code ---

                property_type = row.get('statusText', '')
                if property_type and 'for sale' in property_type:
                    property_type = property_type.split('for sale')[0].strip()

                if pd.notna(row.get('carouselPhotos')):
                    try:
                        if isinstance(row['carouselPhotos'], str):
                            photos = json.loads(row['carouselPhotos'].replace("'", "\""))
                        elif isinstance(row['carouselPhotos'], list):
                            photos = row['carouselPhotos']
                        else:
                            photos = []

                        photo_urls = [photo.get('url') for photo in photos if isinstance(photo, dict) and 'url' in photo]

                    except json.JSONDecodeError:
                        photo_urls = []
                else:
                    photo_urls = []

                # Generate a unique property _id
                property_id = str(ObjectId())

                # Build the flat property record (for PropertyOnSale)
                # Also include latitude and longitude for coordinates
                property_record = {
                    '_id': property_id,
                    'city': city.title(),
                    'neighbourhood': neighbourhood,
                    'address': row.get('address', ''),
                    'price': row.get('unformattedPrice', 0),
                    'thumbnail': row.get('imgSrc', ''),
                    'type': property_type,
                    'bed_number': row.get('beds', 0),
                    'bath_number': row.get('baths', 0),
                    'area': area_value,
                    'photos': photo_urls,
                    'registration_date': registration_date,
                    'disponibility': {
                        'day': open_house_day,
                        'time': open_house_time,
                        'max_attendees': max_attendees
                    },
                    # Temporary field to group by agency
                    'agency_name': agency_name,
                    'latitude': lat,
                    'longitude': lon
                }

                if not pd.isna(description):
                    property_record['description'] = description

                all_properties.append(property_record)

                # Group properties by agency (seller)
                if agency_name not in sellers:
                    sellers[agency_name] = {
                        '_id': str(ObjectId()),
                        'agency_name': agency_name,
                        'email': '',      # to be generated if missing
                        'password': '',   # to be generated if missing
                        'properties': []  # temporary list of properties for this seller
                    }
                sellers[agency_name]['properties'].append(property_record)

    # For each seller, generate email and password if missing
    for seller in sellers.values():
        if not seller['email']:
            if seller['agency_name']:
                seller['email'] = f"{seller['agency_name'].replace(' ', '').lower()}@seller.com"
            else:
                seller['email'] = fake.email()
        if not seller['password']:
            seller['password'] = hash_password(fake.password())

    # Separate properties into:
    # - property_on_sale: with fields _id, city, neighbourhood, address, price, thumbnail and disponibility
    # - sold_property: randomly select some properties and add sell_date (2 days after registration_date)
    sellers_final = []
    properties_on_sale_flat = []  # List of unsold properties for the PropertyOnSale CSV
    for seller in sellers.values():
        properties_list = seller.pop('properties')
        sold_properties = []
        on_sale_properties = []

        # Randomly decide for each property if it is sold (30% probability)
        for prop in properties_list:
            if random.random() < 0.3:
                reg_date = datetime.fromisoformat(prop['registration_date'])
                sell_date = (reg_date + timedelta(days=2)).isoformat()
                sold_prop = {
                    '_id': prop['_id'],
                    'city': prop['city'],
                    'neighbourhood': prop['neighbourhood'],
                    'price': prop['price'],
                    'thumbnail': prop['thumbnail'],
                    'type': prop['type'],
                    'area': prop['area'],
                    'registration_date': prop['registration_date'],
                    'sell_date': sell_date
                }
                sold_properties.append(sold_prop)
            else:
                on_sale_prop = {
                    '_id': prop['_id'],
                    'city': prop['city'],
                    'neighbourhood': prop['neighbourhood'],
                    'address': prop['address'],
                    'price': prop['price'],
                    'thumbnail': prop['thumbnail'],
                    'disponibility': prop['disponibility']
                }
                on_sale_properties.append(on_sale_prop)
                # Also add the property to the global list for the PropertyOnSale CSV
                properties_on_sale_flat.append(prop)

        seller['sold_property'] = sold_properties
        seller['property_on_sale'] = on_sale_properties
        sellers_final.append(seller)

    # Export sellers (ex-agencies) to CSV
    # Convert embedded arrays to JSON strings
    sellers_df = pd.DataFrame(sellers_final)
    sellers_df['property_on_sale'] = sellers_df['property_on_sale'].apply(json.dumps)
    sellers_df['sold_property'] = sellers_df['sold_property'].apply(json.dumps)
    sellers_output = os.path.join(output_folder, 'sellers.csv')
    sellers_df.to_csv(sellers_output, index=False)

    # Export PropertyOnSale (flat) CSV
    # Ensure nested fields (photos and disponibility) are converted to strings
    for prop in properties_on_sale_flat:
        prop['photos'] = json.dumps(prop['photos'])
        prop['disponibility'] = json.dumps(prop['disponibility'])
    properties_df = pd.DataFrame(properties_on_sale_flat)
    # Remove agency_name field from properties
    properties_df.drop(columns=['agency_name'], inplace=True)
    properties_output = os.path.join(output_folder, 'properties_on_sale.csv')
    properties_df.to_csv(properties_output, index=False)

    # Generate properties_on_sale_neo4j.csv file
    neo4j_records = []
    for prop in properties_on_sale_flat:
        lat = prop.get('latitude', '')
        lon = prop.get('longitude', '')
        try:
            price = float(prop.get('price', 0))
        except:
            price = 0.0
        score = random.uniform(0, 100)
        neo4j_records.append({
            'property_on_sale_id': prop['_id'],
            'latitude': lat,
            'longitude': lon,
            'price': price,
            'score': score,
            'type': prop['type'],
            'thumbnail': prop['thumbnail'],
            'neighbourhood': prop['neighbourhood']
        })
    neo4j_df = pd.DataFrame(neo4j_records)
    neo4j_output = os.path.join(output_folder, 'properties_on_sale_neo4j.csv')
    neo4j_df.to_csv(neo4j_output, index=False)

    # Generate buyers based on unsold properties
    generate_buyers(properties_on_sale_flat, output_folder)

# Input/output paths and loading the neighbourhoods file
input_folder = '/drive/MyDrive/Intermediate Corporation/Dati/Zillow/Scraped'
output_folder = '/drive/MyDrive/Intermediate Corporation/Dati/Zillow/Formatted'
neighbourhoods_df = pd.read_csv('/drive/MyDrive/Intermediate Corporation/Dati/Cities and neighbourhoods/neighbourhoods.csv')
neighbourhoods_df = neighbourhoods_df[neighbourhoods_df["name"] != "Unknown"]

print(os.listdir(input_folder))
print(os.listdir(output_folder))
process_files(input_folder)


['properties_new york_ny.csv', 'properties_los angeles_ca.csv', 'properties_chicago_il.csv', 'properties_houston_tx.csv', 'properties_phoenix_az.csv', 'properties_philadelphia_pa.csv', 'properties_san antonio_tx.csv', 'properties_san diego_ca.csv', 'properties_dallas_tx.csv', 'properties_austin_tx.csv', 'properties_jacksonville_fl.csv', 'properties_san jose_ca.csv', 'properties_indianapolis_in.csv', 'properties_san francisco_ca.csv', 'properties_columbus_oh.csv', 'properties_fort worth_tx.csv', 'properties_charlotte_nc.csv', 'properties_oklahoma city_ok.csv', 'properties_detroit_mi.csv']
['sellers.csv', 'properties_on_sale_neo4j.csv', 'properties_on_sale.csv', 'buyers.csv', 'reservations_seller.csv', 'reservations_buyer.csv']
